<a href="https://colab.research.google.com/github/ChenghenChen/Slum/blob/main/030624.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
!pip install segment-geospatial leafmap pillow


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.8/50.8 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 53.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 72.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 262.5/262.5 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import os
from PIL import Image

# 分割图像函数
def split_image(image_path, out_dir, grid_size=10):
    img = Image.open(image_path)
    img_width, img_height = img.size
    tile_width, tile_height = img_width // grid_size, img_height // grid_size

    # 创建输出目录
    os.makedirs(out_dir, exist_ok=True)

    for i in range(grid_size):
        for j in range(grid_size):
            box = (j * tile_width, i * tile_height, (j + 1) * tile_width, (i + 1) * tile_height)
            out_path = os.path.join(out_dir, f'tile_{i}_{j}.png')
            img.crop(box).save(out_path, 'PNG')

# 分割原始图像
original_image_path = '/content/Satpic/Artboard 1 copy 3.png'
split_images_dir = '/content/drive/My Drive/SplitImages'  # 保存分割图像的Google Drive目录
split_image(original_image_path, split_images_dir)


FileNotFoundError: [Errno 2] No such file or directory: '/content/Satpic/Artboard 1 copy 3.png'

In [ ]:
from samgeo import SamGeo

# 初始化 SAMGEO 模型
sam = SamGeo(model_type="vit_h", checkpoint="sam_vit_h_4b8939.pth")

# 检查已经处理过的图像
processed_files = set()
processed_file_path = '/content/drive/My Drive/SplitImages/processed.txt'  # 处理过的图像列表
if os.path.exists(processed_file_path):
    with open(processed_file_path, 'r') as f:
        processed_files = set(f.read().splitlines())

# 处理每个小图像
for i in range(10):
    for j in range(10):
        tile_path = f'{split_images_dir}/tile_{i}_{j}.png'
        if tile_path in processed_files:
            continue  # 跳过已处理的图像

        # 分割并保存
        segmented_path = f'{split_images_dir}/segmented_tile_{i}_{j}.png'
        sam.generate(tile_path, segmented_path, batch=True, foreground=True, erosion_kernel=(3, 3), mask_multiplier=255)

        # 记录处理过的图像
        with open(processed_file_path, 'a') as f:
            f.write(f"{tile_path}\n")

        # 可选：展示第一张图的分割结果
        if i == 0 and j == 0:
            from PIL import Image
            import matplotlib.pyplot as plt

            # 显示第一张分割后的图像
            segmented_img = Image.open(segmented_path)
            plt.figure(figsize=(8, 8))
            plt.imshow(segmented_img)
            plt.title("Segmented Tile 0_0")
            plt.show()
